<a href="https://colab.research.google.com/github/MarcusKLY/AI-Stock-Prediction/blob/main/Price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import platform
platform.platform()

In [ ]:
import sys
sys.version_info

In [ ]:
!nvidia-smi -L
!cat /proc/cpuinfo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import time

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# load data
start_time = time.time()
company = 'MS'
epochs = 25
batch_size = 32
units = 60
prediction_days = 60
start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()


#################################################

data = web.DataReader(company, 'stooq', start, end)
data = data[::-1]

# prepare data for model
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))



x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) #reshape (x_train, into.....)

# Build The Model
model = Sequential()

# 1 LSTM layer, 1 drop out layer,1 LSTM layer, 1 drop out layer, finally a dense layer
start_time = time.time()
model.add(LSTM(units=units, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=units, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=units)) #no need to return anymore
model.add(Dropout(0.2))
model.add(Dense(units=1)) #prediction of the next closing value

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size) #epochs means the system see the data 24 times #batch_sizes mean the model see 32 data at once
time = ("%s" % (time.time() - start_time))

model.save('model1')
# ''' Test The Model Accuracy on Existing Data '''

# Load Test Data #use model that the program never see before


test_data = web.DataReader(company, 'stooq', test_start, test_end)
test_data = test_data[::-1]
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values # what model is going to see
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

# make predictions on test data

x_test = []

for x in range(prediction_days, len(model_inputs)): #+1 is the newest one
    x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_text = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)


# predict next stock market day

real_data = [model_inputs[len(model_inputs) - prediction_days:len(model_inputs + 1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f'Prediction: {prediction}')
rmse = np.sqrt((np.mean(predicted_prices)-np.mean(actual_prices))**2)

#rmse = np.sqrt(np.mean(np.mean(predicted_prices-actual_prices)**2))
print(f'rms error: {rmse}')
print(f'time(secs): {time}')

# Plot the test Prediction

plt.plot(actual_prices, color='b', label=f'Actual {company} Price')
plt.plot(predicted_prices, color='r', label=f'Predicted {company} Price')
plt.title(f'{company} Share Price')
plt.xlabel('Time')
plt.ylabel(f'{company} Share Price')
plt.legend()
plt.savefig(f'company={company} units={units} epochs={epochs} batch_size={batch_size} tomorrow_prediction={prediction} rmse={rmse} prediction days={prediction_days} model training time(sec)={time}.png', dpi=1000)